In [3]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.types import TimestampType

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

geoEventProcessing = spark.readStream\
                          .format("socket")\
                          .option("host", "localhost")\
                          .option("port", 9999)\
                          .load()\
                          .select(split("value",'\|').alias("fields"))\
                          .withColumn("timestamp_str",col("fields").getItem(0))\
                          .withColumn("timestamp",col("timestamp_str").cast(TimestampType()))\
                          .withColumn("driverName",col("fields").getItem(4))\
                          .withColumn("violation", col("fields").getItem(7))\
                          .where("violation!='Normal'")\
                          .groupBy("driverName", "violation")\
                          .count()\
                          .orderBy(col("count").desc())\
                          .limit(5)\
                          .writeStream\
                          .format("console")\
                          .outputMode("complete")\
                          .trigger(processingTime='10 seconds')\
                          .start()\
                          .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+---------+-----+
|driverName|violation|count|
+----------+---------+-----+
+----------+---------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|Unsafe tail distance|    3|
|       Joe Witt|      Lane Departure|    3|
|       Joe Witt|Unsafe tail distance|    3|
|Suresh Srinivas|      Lane Departure|    3|
|       Joe Witt|           Overspeed|    1|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|      Lane Departure|    5|
|       Joe Witt|           Overspeed|    4|
|Suresh Srinivas|Unsafe tail distance|    3|
|       Joe Witt|      Lane Departure|    3|
|       Joe Witt|Unsafe tail distance|    3|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|      Lane Departure|    6|
|Suresh Srinivas|Unsafe tail distance|    4|
|       Joe Witt|      Lane Departure|    4|
|       Joe Witt|           Overspeed|    4|
|       Joe Witt|Unsafe tail distance|    3|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|      Lane Departure|    8|
|Suresh Srinivas|Unsafe tail distance|    5|
|       Joe Witt|           Overspeed|    5|
|       Joe Witt|      Lane Departure|    4|
|       Joe Witt|Unsafe tail distance|    4|
+---------------+--------------------+-----+



ERROR:root:KeyboardInterrupt while sending command.              (25 + 2) / 200]
Traceback (most recent call last):
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 